# Алгоритмы интеллектуальной обработки больших объемов данных
## Домашнее задание №2 - Дерево решений


**Общая информация**

**Срок сдачи:** до 1 ноября 2018, 06:00 
**Штраф за опоздание:** -2 балла после 06:00 1 ноября, -4 балла после 06:00 8 ноября, -6 баллов после 06:00 15 ноября

При отправлении ДЗ указывайте фамилию в названии файла   


Присылать ДЗ необходимо в виде ссылки на свой github репозиторий в slack @alkhamush
Необходимо в slack создать таск в приватный чат:   
/todo Фамилия Имя *ссылка на гитхаб* @alkhamush   
Пример:   
/todo Ксения Стройкова https://github.com/stroykova/spheremailru/stroykova_hw2.ipynb @alkhamush   

Используйте данный Ipython Notebook при оформлении домашнего задания.

###### Задание 1 (4 балла)
Разберитесь в коде MyDecisionTreeClassifier, который уже частично реализован. В комментариях, где написано "Что делает этот блок кода?", ответьте на этот вопрос. Допишите код там, где написано "Ваш код". Ваша реализация дерева должна работать по точности не хуже DecisionTreeClassifier из sklearn. Точность проверяется на wine и Speed Dating Data.

###### Задание 2 (2 балла)
Добиться скорости работы fit такой, чтобы она была медленнее sklearn не более чем в 10 раз. Скорость проверяем на  wine и Speed Dating Data. Для ускорения используем только numpy.

###### Задание 3 (2 балла)
Добавьте функционал, который определяет значения feature importance. Выведите 10 главных фичей под пунктом Задание 3 (уже написано ниже) для MyDecisionTreeClassifier и DecisionTreeClassifier так, чтобы сразу были видны выводы и по MyDecisionTreeClassifier, и по DecisionTreeClassifier. Используем данные Speed Dating Data.

###### Задание 4 (2 балла)
С помощью GridSearchCV или RandomSearchCV подберите наиболее оптимальные параметры для случайного леса (Выберете 2-3 параметра). Используем данные Speed Dating Data. Задание реализуйте под пунктом Задание 5 (уже написано ниже)


**Штрафные баллы:**

1. Невыполнение PEP8 -1 балл
2. Отсутствие фамилии в имени скрипта (скрипт должен называться по аналогии со stroykova_hw2.ipynb) -1 балл
3. Все строчки должны быть выполнены. Нужно, чтобы output команды можно было увидеть уже в git'е. В противном случае -1 балл
4. При оформлении ДЗ нужно пользоваться данным файлом в качестве шаблона. Не нужно удалять и видоизменять написанный код и текст. В противном случае -1 балл

In [4]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

from sklearn.datasets import load_wine
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, f1_score
from sklearn.model_selection import KFold, train_test_split, GridSearchCV, RandomizedSearchCV
from sklearn.tree import DecisionTreeClassifier

%matplotlib inline
%load_ext pycodestyle_magic

The pycodestyle_magic extension is already loaded. To reload it, use:
  %reload_ext pycodestyle_magic


In [304]:
#%%pycodestyle


class MyDecisionTreeClassifier:
    NON_LEAF_TYPE = 0
    LEAF_TYPE = 1

    def __init__(self, min_samples_split=2, max_depth=None,
                 sufficient_share=1.0, criterion='gini', max_features=None):
        self.tree = dict()
        self.min_samples_split = min_samples_split
        self.max_depth = max_depth
        self.sufficient_share = sufficient_share
        self.num_class = -1
        self.feature_importances_ = None
        if criterion == 'gini':
            self.G_function = self.__gini
            self.gain = self.__gini_gain
        elif criterion == 'entropy':
            self.G_function = self.__entropy
            self.gain = self.__entropy_gain
        elif criterion == 'misclass':
            self.G_function = self.__misclass
            self.gain = self.__misclass_gain
        else:
            print('invalid criterion name')
            raise

        if max_features == 'sqrt':
            self.get_feature_ids = self.__get_feature_ids_sqrt
        elif max_features == 'log2':
            self.get_feature_ids = self.__get_feature_ids_log2
        elif max_features is None:
            self.get_feature_ids = self.__get_feature_ids_N
        else:
            print('invalid max_features name')
            raise

    def __gini(self, l_c, l_s, r_c, r_s):
        return 1 - ((l_c ** 2 / l_s).sum(axis=1) + (r_c ** 2 / r_s).sum(axis=1)) / (l_s + r_s)[0]

    def __entropy(self, l_c, l_s, r_c, r_s):
        return -((l_c * (l_c / l_s).log2()).nansum(axis=1) + (r_c * (r_c / r_s).log2()).nansum(axis=1))/(l_s + r_s)[0]

    def __misclass(self, l_c, l_s, r_c, r_s):
        return 1 - (l_c.max(axis=1) + r_c.max(axis=1)) / (l_s + r_s)[0]

    def __gini_gain(self, y):
        return 1 - ((np.bincount(y).astype(float) / y.size) ** 2).sum()

    def __entropy_gain(self, y):
        return -((np.bincount(y).astype(float) / y.size) * (np.bincount(y).astype(float) / y.size).log2()).nansum()

    def __misclass_gain(self, y):
        return 1 - (np.bincount(y).astype(float) / y.size).max()

    def __get_feature_ids_sqrt(self, n_feature):
        feature_ids = range(n_feature)
        np.random.shuffle(feature_ids)
        return np.array(feature_ids[:max(1, np.int(np.sqrt(n_feature)))])

    def __get_feature_ids_log2(self, n_feature):
        feature_ids = range(n_feature)
        np.random.shuffle(feature_ids)
        return np.array(feature_ids[:max(1, np.int(np.log2(n_feature)))])

    def __get_feature_ids_N(self, n_feature):
        return np.arange(n_feature)

    def __sort_samples(self, x, y):
        sorted_idx = x.argsort()
        return x[sorted_idx], y[sorted_idx]

    def __div_samples(self, x, y, feature_id, threshold):
        left_mask = x[:, feature_id] > threshold
        right_mask = ~left_mask
        return x[left_mask], x[right_mask], y[left_mask], y[right_mask]

    def __find_threshold(self, x, y):
        # Что делает этот блок кода?
        '''
        Параллельная сортировка входных признаков и ответов(по признакам),
        присвоение количества классов,
        считаем количество ответов, которые будут удалены после прохождения через порог
        (их сумма в будущем должна быть равна min_samples_split)
        
        '''
        sorted_x, sorted_y = self.__sort_samples(x, y)
        class_number = self.num_class
        cut_size = np.int(self.min_samples_split / 2 - 1)

        # Что делает этот блок кода?
        '''        
        Находим все те id, в которых значение y меняются при прохождении через порог,
        в каждом разбиении не менее min_samples_split элементов,
        случай с отсутсвием таких инедксов исключается
        '''
        if cut_size == 0:
            splitted_sorted_y = sorted_y
        else:
            splitted_sorted_y = sorted_y[cut_size:-cut_size]
        r_border_ids = np.where(splitted_sorted_y[:-1] !=
                                splitted_sorted_y[1:])[0] + (cut_size + 1)

        if len(r_border_ids) == 0:
            return np.inf, None

        # Что делает этот блок кода?
        '''
        Записываем в one_hot_code 1 там, где класс меняется,
        в class_increments пишем сколько элементов в каком классе лежит
        '''
        eq_el_count = r_border_ids - np.append(np.array([cut_size]),
                                               r_border_ids[:-1])
        one_hot_code = np.zeros((r_border_ids.shape[0], class_number))
        one_hot_code[np.arange(r_border_ids.shape[0]),
                     sorted_y[r_border_ids - 1]] = 1
        class_increments = one_hot_code * eq_el_count.reshape(-1, 1)
        class_increments[0] = class_increments[0] + \
            np.bincount(sorted_y[:cut_size], minlength=class_number)

        # Что делает этот блок кода?
        '''
        Считаем количество элементов, принадлежащих каждому классу слева и справа от границы,
        считаем суммарное количество эдементов слева и справа от границы
        '''
        l_class_count = np.cumsum(class_increments, axis=0)
        r_class_count = np.bincount(sorted_y,
                                    minlength=class_number) - l_class_count
        l_sizes = r_border_ids.reshape(l_class_count.shape[0], 1)
        r_sizes = sorted_y.shape[0] - l_sizes

        # Что делает этот блок кода?
        '''
        Высчитываем метрику неопределнности,
        высчитываем ее минимальный порог
        '''
        gs = self.G_function(l_class_count, l_sizes, r_class_count, r_sizes)
        idx = np.argmin(gs)

        # Что делает этот блок кода?
        '''
        Возвращаем оптимальное значение порога,
        '''
        left_el_id = l_sizes[idx][0]
        return gs[idx], (sorted_x[left_el_id-1] + sorted_x[left_el_id]) / 2.0

    def __fit_node(self, x, y, node_id, depth, pred_f=-1):
        y_bins = np.bincount(y)
        y_bins_armx = y_bins.argmax()
        sz = y_bins.astype(float) / y.size

        if (self.max_depth is not None and depth >= self.max_depth) or y.size < self.min_samples_split or\
                y_bins[y_bins_armx] >= y.size * self.sufficient_share:
            self.tree[node_id] = (self.LEAF_TYPE, y_bins_armx, sz)
        else:
            feature_ids = self.get_feature_ids(x.shape[1])
            min_impur = float('inf')
            
            threshes = np.array([self.__find_threshold(x[:, i], y) for i in feature_ids])
            
            if(np.amin(threshes[:, 0]) < min_impur):
                min_impur = np.amin(threshes[:, 0])
                best_feature_id = np.argmin(threshes[:, 0])
                best_thresh = threshes[best_feature_id, 1]
            
            if best_thresh is None:
                self.tree[node_id] = (self.LEAF_TYPE, y_bins_armx, sz)
            else:
                x_l, x_r, y_l, y_r = self.__div_samples(x, y, best_feature_id, best_thresh)
                if x_l.size == 0 or x_r.size == 0:
                    self.tree[node_id] = (self.LEAF_TYPE, y_bins_armx, sz)
                else:
                    self.tree[node_id] = (self.NON_LEAF_TYPE, best_feature_id, best_thresh)
                    self.feature_importances_[best_feature_id] += self.gain(y) * y.size -\
                     (self.gain(y_l) * y_l.size + self.gain(y_r) * y_r.size)
                    self.__fit_node(x_l, y_l, 2 * node_id + 1, depth + 1, pred_f)
                    self.__fit_node(x_r, y_r, 2 * node_id + 2, depth + 1, pred_f)

    def fit(self, x, y):
        self.feature_importances_ = np.zeros(x.shape[1])
        self.num_class = np.unique(y).size
        self.__fit_node(x, y, 0, 0)

    def __predict_class(self, x, node_id):
        node = self.tree[node_id]
        if node[0] == self.__class__.NON_LEAF_TYPE:
            _, feature_id, threshold = node
            if x[feature_id] > threshold:
                return self.__predict_class(x, 2 * node_id + 1)
            else:
                return self.__predict_class(x, 2 * node_id + 2)
        else:
            return node[1]

    def __predict_probs(self, x, node_id):
        node = self.tree[node_id]
        if node[0] == self.__class__.NON_LEAF_TYPE:
            _, feature_id, threshold = node
            if x[feature_id] > threshold:
                return self.__predict_probs(x, 2 * node_id + 1)
            else:
                return self.__predict_probs(x, 2 * node_id + 2)
        else:
            return node[2]

    def predict(self, X):
        return np.array([self.__predict_class(x, 0) for x in X])

    def predict_probs(self, X):
        return np.array([self.__predict_probs(x, 0) for x in X])

    def fit_predict(self, x_train, y_train, predicted_x):
        self.fit(x_train, y_train)
        return self.predict(predicted_x)

In [305]:
my_clf = MyDecisionTreeClassifier(min_samples_split=2)
clf = DecisionTreeClassifier(min_samples_split=2)

In [306]:
wine = load_wine()
X_train, X_test, y_train, y_test = train_test_split(wine.data, wine.target, test_size=0.1, stratify=wine.target)

In [307]:
r_border_ids = [7, 9, 11]
np.append(np.array([6]),r_border_ids[:-1])

array([6, 7, 9])

## Проверка скорости работы на wine

In [308]:
%time clf.fit(X_train, y_train)

Wall time: 2 ms


DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best')

In [309]:
%time my_clf.fit(X_train, y_train)

Wall time: 20 ms


## Проверка качества работы на wine

In [310]:
f1_score(y_pred=clf.predict(X_test), y_true=y_test, average='macro')

0.8917748917748917

In [311]:
f1_score(y_pred=my_clf.predict(X_test), y_true=y_test, average='macro')

0.8917748917748917

## Подготовка данных Speed Dating Data 

In [343]:
df = pd.read_csv('Speed Dating Data.csv', encoding='cp1251')

In [344]:
def prepare(df):
    df = df.iloc[:, :97]
    drop_list = ['id', 'idg', 'condtn', 'round', 'position', 'positin1',
                 'order', 'partner', 'age_o', 'race_o', 'pf_o_att', 'pf_o_sin',
                  'pf_o_int', 'pf_o_fun', 'pf_o_amb', 'pf_o_sha',
                  'dec_o', 'attr_o',
                  'sinc_o', 'intel_o', 'fun_o', 'amb_o', 'shar_o', 'like_o',
                  'prob_o', 'met_o', 'field', 'undergra',
                  'imprelig', 'imprace',
                  'from', 'zipcode', 'career', 'sports',
                  'tvsports', 'exercise',
                  'dining', 'museums',
                  'art', 'hiking', 'gaming', 'clubbing', 'reading', 'tv',
                  'theater', 'movies', 'concerts', 'music', 'shopping',
                  'yoga', 'expnum', 'wave']
    df = df.drop(drop_list, axis=1)
    df = df.dropna(subset=['age'])
    df.loc[:, 'field_cd'] = df.loc[:, 'field_cd'].fillna(0)
    pd.get_dummies(df, columns=['field_cd'], prefix='field_cd', prefix_sep='=')
    pd.get_dummies(df, columns=['race'], prefix='race', prefix_sep='=')
    df.loc[:, 'mn_sat'] =\
        df.loc[:, 'mn_sat'].str.replace(',', '').astype(np.float)
    df.loc[:, 'mn_sat'] = df.mn_sat.fillna(-999)
    df.loc[:, 'tuition'] =\
        df.loc[:, 'tuition'].str.replace(',', '').astype(np.float)
    df.loc[:, 'tuition'] = df.tuition.fillna(-999)
    df.loc[:, 'income'] =\
        df.loc[:, 'income'].str.replace(',', '').astype(np.float)
    df.loc[:, 'income'] = df.loc[:, 'income'].fillna(-999)
    df = df.dropna(subset=['date'])
    df.loc[:, 'career_c'] = df.loc[:, 'career_c'].fillna(0)
    df.loc[:, 'temp_totalsum'] =\
        df.loc[:, ['attr1_1', 'sinc1_1', 'intel1_1',
                   'fun1_1', 'amb1_1', 'shar1_1']].sum(axis=1)
    df.loc[:, ['attr1_1', 'sinc1_1', 'intel1_1', 'fun1_1', 'amb1_1',
               'shar1_1']] = (df.loc[:, ['attr1_1', 'sinc1_1', 'intel1_1',
                                         'fun1_1', 'amb1_1', 'shar1_1']].T /
                              df.loc[:, 'temp_totalsum'].T).T * 100
    df.loc[:, 'temp_totalsum'] = df.loc[:, ['attr2_1', 'sinc2_1', 'intel2_1',
                                            'fun2_1', 'amb2_1', 'shar2_1']]\
                                   .sum(axis=1)
    df.loc[:, ['attr2_1', 'sinc2_1', 'intel2_1', 'fun2_1', 'amb2_1',
               'shar2_1']] = (df.loc[:, ['attr2_1', 'sinc2_1', 'intel2_1',
                                         'fun2_1', 'amb2_1', 'shar2_1']].T /
                              df.loc[:, 'temp_totalsum'].T).T * 100
    df = df.drop(['temp_totalsum'], axis=1)
    for i in [4, 5]:
        feat = ['attr{}_1'.format(i), 'sinc{}_1'.format(i),
                'intel{}_1'.format(i), 'fun{}_1'.format(i),
                'amb{}_1'.format(i), 'shar{}_1'.format(i)]
        if i != 4:
            feat.remove('shar{}_1'.format(i))
        df = df.drop(feat, axis=1)

    df_male = df.query('gender == 1').drop_duplicates(subset=['iid', 'pid'])\
                .drop(['gender'], axis=1).dropna()
    df_female = df.query('gender == 0').drop_duplicates(subset=['iid'])\
                  .drop(['gender', 'match', 'int_corr', 'samerace'], axis=1)\
                  .dropna()
    df_female.columns = df_female.columns + '_f'
    df_female = df_female.drop(['pid_f'], axis=1)
    df_pair = df_male.join(df_female.set_index('iid_f'), on='pid', how='inner')
    df_pair = df_pair.drop(['iid', 'pid'], axis=1)
    X = df_pair.iloc[:, 1:].values
    y = df_pair.iloc[:, 0].values
    features = df_pair.columns[1:]
    return X, y, features

In [345]:
df = pd.read_csv('Speed Dating Data.csv', encoding='cp1251')
X, y, features = prepare(df)
X_train, X_test, y_train, y_test = \
        train_test_split(X, y, test_size=0.1)
my_clf = MyDecisionTreeClassifier(min_samples_split=2)
clf = DecisionTreeClassifier(min_samples_split=2)

## Проверка скорости работы на Speed Dating Data 

In [346]:
%time clf.fit(X_train, y_train)

Wall time: 109 ms


DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best')

In [347]:
%time my_clf.fit(X_train, y_train)

Wall time: 1.19 s


## Проверка качества работы на Speed Dating Data

In [348]:
f1_score(y_pred=clf.predict(X_test), y_true=y_test, average='macro')

0.5721726190476191

In [349]:
f1_score(y_pred=my_clf.predict(X_test), y_true=y_test, average='macro')

0.582120321488766

## Задание 3

In [350]:
sr = pd.Series(data=my_clf.feature_importances_, index=features)
sr.div(sr.sum()).sort_values(ascending=False).head(10)

int_corr     0.192641
goal_f       0.050031
age_f        0.049066
income_f     0.041606
date         0.038291
attr3_1      0.036085
age          0.032539
attr2_1_f    0.030933
shar1_1      0.029657
income       0.028125
dtype: float64

In [351]:
pd.Series(data=clf.feature_importances_,index=features).sort_values(ascending=False).head(10)

int_corr      0.086086
amb1_1        0.034178
sinc1_1       0.027430
intel1_1_f    0.027156
mn_sat_f      0.025230
age           0.024482
age_f         0.024235
attr2_1       0.023998
income_f      0.023896
attr1_1_f     0.023852
dtype: float64

## Задание 4

In [285]:
clf = RandomForestClassifier()
var_param = {
    "min_samples_split": [i for i in range(2, 25)],
    "max_depth": [i for i in range(1, 100)] + [None],
    "criterion": ["gini", "entropy"],
    "n_estimators": [i for i in range(1, 35)]
}
iters = 30
random_search = RandomizedSearchCV(clf, param_distributions=var_param, n_iter=iters)
random_search.fit(X, y)
print(random_search.best_params_)

C:\Users\USER\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:1943: FutureWarning: You should specify a value for 'cv' instead of relying on the default value. The default value will change from 3 to 5 in version 0.22.
  warnings.warn(CV_WARNING, FutureWarning)


{'n_estimators': 30, 'min_samples_split': 20, 'max_depth': 7, 'criterion': 'entropy'}
